# ACL Overview
In the Windows operating system, everything is an object - that is, everything is represented by data structures in memory, that contain members that describe the nature of the data the structure is meant to represent. Obvious things such as files and registry keys are objects, but also more abstract things such as processes and threads are also objects.

Active Directory is no different - AD is primarily a database of components that represent the various identities and relationships in a Windows network. Each object or component in an AD environment is a user, a group, a computer, a printer, a service account, or even a container for any of these.

Again, as in the Windows operating system, Active Directory objects control who has access via Access Control Lists or ACLs (specifically the **D**iscretionary **A**ccess **C**ontrol List or _DACL_), with each Access Control Entry (or ACE) in the ACL specifying an identity (formally called a security principal) and what specific access rights that identity can excise on the object.

Access rights are incredibly powerful - the demonstration below is not exhaustive, but it will be become apparent how destructive a misconfiured ACE on the wrong object can be. Notably however, many of the attacks that are enabled by these access rights are detailed in their own notebooks, making understanding what objects have rights over other objects one of the fundamental tasks in reconaissance, and in understanding the attack paths through an AD environment.  

This lab will focus on an attack path that relies entirely on misconfigured ACLs. While this is a contrived example, it will demonstrate how these attacks paths work, and lay important foundations for other labs. If you skip any lab, do not skip this one! 

For this lab, we will assume we have access to the account of a user called "John Dee" by some initial access technique.

In [4]:
$domainController = "dc01.aclabuse.lab"
$userName = "john.dee"
$creds = "johnsPass01"
$domainName = "aclabuse.lab"
$domain = New-Object DirectoryServices.DirectoryEntry("LDAP://$domainController","$domainName\$userName", $creds)
$searcher = New-Object System.DirectoryServices.DirectorySearcher($domain)

$searcher.Filter = "(&(objectCategory=group)(objectClass=group)(cn=*admin*))"
$groups = $searcher.FindOne()

foreach ($group in $groups) {
    $groupDN = $group.Properties["distinguishedname"][0]
    $groupName = $group.Properties["cn"][0]
    # Get the DACL of the group
    $groupEntry = $group.GetDirectoryEntry()
    $groupACL = $groupEntry.ObjectSecurity.GetAccessRules($true, $true, [System.Security.Principal.NTAccount])
    foreach ($ace in $groupACL) {
        if ($ace.IdentityReference == "$domainName\$userName") {
        Write-Host "DN: $groupDN Name: $groupName"
        Write-Host "AccessControlType: $($ace.AccessControlType)"
        Write-Host "AccessControlType: $($ace.ActiveDirectory)"
    }

}


IdentityReference: NT AUTHORITY\Authenticated Users
AccessControlType: Allow
IdentityReference: NT AUTHORITY\SYSTEM
AccessControlType: Allow
IdentityReference: BUILTIN\Administrators
AccessControlType: Allow
IdentityReference: S-1-5-21-134974639-3553680865-2577818021-512
AccessControlType: Allow
IdentityReference: S-1-5-21-134974639-3553680865-2577818021-519
AccessControlType: Allow
IdentityReference: S-1-5-21-134974639-3553680865-2577818021-1106
AccessControlType: Allow
IdentityReference: Everyone
AccessControlType: Allow
IdentityReference: NT AUTHORITY\SELF
AccessControlType: Allow
IdentityReference: NT AUTHORITY\SELF
AccessControlType: Allow
IdentityReference: S-1-5-32-554
AccessControlType: Allow
IdentityReference: S-1-5-32-554
AccessControlType: Allow
IdentityReference: S-1-5-32-554
AccessControlType: Allow
IdentityReference: S-1-5-32-554
AccessControlType: Allow
IdentityReference: S-1-5-32-554
AccessControlType: Allow
IdentityReference: S-1-5-32-554
AccessControlType: Allow
Ident

## WriteProperty
The first right to examine is `WriteProperty`. This right, as the name implies, grants a principal the capability to modify properties on another object.

## WriteDACL

## GenericWrite

## GenericAll